#Домашнє завдання до тем apply(), groupby(), pivot_table()

В цьому домашньому завданні продовжуємо працювати з набором даних `supermarket_sales.csv`.

0. Імпортуйте бібліотеку pandas та зчитайте дані у змінну `df` типу `pandas.DataFrame`.

In [2]:
import pandas as pd

In [4]:
df = pd.read_csv('../Data/supermarket_sales.csv')

In [111]:
df.columns

Index(['Invoice ID', 'Branch', 'City', 'Customer type', 'Gender',
       'Product line', 'Unit price', 'Quantity', 'Tax 5%', 'Total', 'Date',
       'Time', 'Payment', 'cogs', 'gross margin percentage', 'gross income',
       'Rating', 'Day_of_week'],
      dtype='object')

1. Дослідимо, який філіал супермаркету ('Branch') є найприбутковішим. Для цього знайдіть сумарний прибуток за кожним філіалом і виявіть, який філіал має найвищий.

In [62]:
branches = df.groupby('Branch')
branches_income = branches[['gross income', 'City']].agg({'gross income':'sum', 'City': 'first'})
branches_income

,gross income,City
Branch,,
A,5057.1605,Yangon
B,5057.0320,Mandalay
C,5265.1765,Naypyitaw


2. В якому місті знайходиться філіал з найвищим прибутком? Може в тому місці нам розмітисти ще один магазин.  
Знайдіть відповідь за допомогою функціоналу Pandas.

За попередніми розрахунками знаємо, що в Naypyitaw живе більш платоспроможне населення. Люди частіше купують товари з дорогого сегменту, середній чек вищий. 
Про ще один магазин:
цікаво, але як розраховують доцільність ще одного магазину? може ті самі клієнти просто розділяться між двома? чи так не буває?

3.1. Створіть зводну таблицю, яка покаже, скільки покупок (інвойсів) було зроблено в кожній з філій (`Branch`) за різними категоріями товарів. Запишіть таблицю в змінну `invoices_by_category` і виведіть змінну на екран.
Ця таблиця допоможе проаналізувати, в якій філії купують найбільше товарів кожної з категорій.

In [120]:
invoices_by_category = pd.pivot_table(df, index='Branch', columns='Product line', values='cogs', aggfunc='count')
invoices_by_category
# одразу зізнаюсь, що очікуваний результат вийшов в мене не прям, щоб свідомо. Спочатку я хотіла рахувати скільки А, В і С
# і таким чином порахувати покупки. Та count не хотів рахувати str, я це зрозуміла і щоб подивитись на якому я світі
# поставила рандомну цифрову колонку, і зʼясувала,що так теж можна. Але прям до кінця я не розібралась чому.

Product line,Electronic accessories,Fashion accessories,Food and beverages,Health and beauty,Home and lifestyle,Sports and travel
Branch,,,,,,
A,60,51,58,47,65,59
B,55,62,50,53,50,62
C,55,65,66,52,45,45


Очікуваний результат:

![](https://drive.google.com/uc?export=view&id=1rueAdko6S3UxIHGtojetTxlES-EyM6Yb)

3.2. Викристовуючи змінну `invoices_by_category` дайте відповідь програмно (тобто значення треба не просто знайти очима, а вивести за допомогою коду), в якому філіалі магазину (`Branch`) найбільше інвойсів із покупкою товарів категорії "Електронні аксесуари" (`Electronic accessories`)?


In [87]:
max_val = invoices_by_category['Electronic accessories'].max()
invoices_by_category[invoices_by_category['Electronic accessories'] == max_val].index

Index(['A'], dtype='object', name='Branch')

У філіалі А.

4-6. **Творче завдання на розвиток аналітичного мислення**

Крок 1. Сформулюйте ТРИ питання (гіпотези) до наявних даних, які допомогли б вам зрозуміти, які користувачі що, де та коли найбільше/найменше купують, аби дати на основі цих гіпотез рекомендації бізнесу. Звісно питання мають бути не тими, на які ми вже відповіли в завданнях модулю.

Крок 2. Знайдіть відповіді на свої питання з допомогою функціоналу pandas.

Крок 3. Напишіть, як відповідь на це питання може бути використана для прийняття бізнес рішень.   
   
 Питання можуть бути будь-якої складності, але їх має бути 3. Кожне питання оцінюється як 1 завдання. Без виконання цього завдання ДЗ не приймається. Якщо є питання щодо виконання - пишіть у чат 🙌


4. Чи впливає день тижня на кільскість покупок?

In [88]:
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')

In [89]:
df['Day_of_week'] = df['Date'].dt.day_name()

In [97]:
days_and_purchases = df.groupby('Day_of_week').size().reset_index(name='Num_of_purch')
days_and_purchases.sort_values(by='Num_of_purch', ascending=False)

,Day_of_week,Num_of_purch
2,Saturday,164
5,Tuesday,158
6,Wednesday,143
0,Friday,139
4,Thursday,138
3,Sunday,133
1,Monday,125


Результати:
- субота - найприбутковіший день(що не суперчить логіці), тож доцільно проводити якісь розпродажі чи акції саме в ций вихідний і на всі категорії товарів, бо люди приходять сімʼями (друзі тощо).
  
- трохи дивно за пʼятницю.. але може всі по барах із колегами-друзями тому і не дивно зовсім? в пятницю, щоб привабити клієнтів, має бути якась розвага, або напої в подарунок, для тих, кто купляє піцу та йде додому чи в гості.

- понеділок - день важкий. Можна просто відчепитися від людей і не заманювати їх щось купити. А можна, типу "всі козли, а ти бубочка. ходи сюди обійму. дивись, ось тобі горнятко кави і смаколик. легше? а тепер диви, шо в нас тут є:....."

5. Хто здійснює покупок більше: лояльні клієнти (з клієнтською карткою) чи інші? Чи є різниця по містах?

In [98]:
city_grouped = df.groupby('City')

In [102]:
city_grouped['Customer type'].value_counts(normalize=True)

City       Customer type
Mandalay   Normal           0.503012
           Member           0.496988
Naypyitaw  Member           0.515244
           Normal           0.484756
Yangon     Normal           0.508824
           Member           0.491176
Name: proportion, dtype: float64

Результат:
Клієнтами магазинів приблизно порівну є лояльні і звичайні.

- можливо, програма лояльності не є цікавою, тому люди не бачать сенсу брати чергову картку, накопичувати бонуси тощо. Це питання можна досліджувати на основі даних про лояльних клієнтів, дивитись як часто вони купуть персональні рекомендовані позиції
- зробити коротке опитування не членів програми лояльності безпосередньо в магазині і спитати чому, навряд половина людей разові клієнти проїздом.
- чи є процес реєстрації легким? (особистий приклад: я відкладала отримання картки лояльності супермаркета півтора роки тому, що до реєстрації було 8-9 кроків, які всі написані нідерландською, а вибору мови в застосунку немає)
- думаю, цей показник вартий уваги і вартий того, щоб збільшувати відсоток лояльних клієнтів тому, що вивчення поведінки клієнта дає більше інформації, в якому напрямку розвиватись, які нови пропозиції зацікавлять тощо.


6. Чи впливає гендерний розподіл на кількість покупок в категорії і дохід?

In [107]:
q = pd.pivot_table(df, index='Gender', columns='Product line', values='Total', aggfunc='sum')

In [108]:
q.round(2)

Product line,Electronic accessories,Fashion accessories,Food and beverages,Health and beauty,Home and lifestyle,Sports and travel
Gender,,,,,,
Female,27102.0,30437.0,33171.0,18561.0,30037.0,28575.0
Male,27236.0,23868.0,22974.0,30633.0,23825.0,26548.0


In [118]:
q = (df.Gender == "Male") & (df['Product line'] == "Health and beauty")
df[q].Total.sum()

# це я себе перевірила

np.float64(30632.7525)

Результат:
співвідношення по доходу виявилось неочікуваним у категорії Health and beauty. Я дивуюся і чесно не розумію як так?!

Висновки:
- можливо, асортимент Health and beauty не релевантний потребам клієнток. А те, що є, здебільшого беруть чоловіки, бо менш прискіпливі у виборі.
Ця категорія закривається клієнтками в іншому магазині, можливо спеціалізованому. Актуалізувати асортимент відповідно до тенденцій. Провести прицільну рекламну кампанію і акції, щоб розказати клієнткам про новинки, і як же зручно, що нікуде більше йти не треба.
- категорія Food and beverages не здивувала. Можна пробувати заохочувати до покупок чоловіків, але чи теба?😏